In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime



### __Computer vision Task 1__

Task: Interactive Image Processing with Mouse  Events and Time Display. 

Goal: Write a program that allows the user to interactively  perform various image processing operations on an image using  mouse events and keyboard shortcuts.
<br /> 
The program should also  display the current time on the frame. 
<br />
The program should  perform the following tasks:

In [2]:
import cv2
import numpy as np
from datetime import datetime

class ImageProcessor:
    def __init__(self, image_path):

        self.image_path = image_path # the path to the image
        self.image = None # the image to be processed
        self.history = [] # list of tuples (operation, image)
        self.original_image = None # the original image

    def read_image(self):
        ''' Read the image from the path '''
        self.image = cv2.imread(self.image_path)
        self.previous_image = self.image.copy()
        self.original_image = self.image.copy() 

    def mouse_callback(self, event, x, y, flags=None, param=None):
        ''' Define a mouse callback function that responds to mouse events. '''

        # if the left button was clicked, draw a rectangle
        if event == cv2.EVENT_LBUTTONDOWN:
            cv2.rectangle(self.image, (x, y), (x + 100, y + 100), (0, 255, 0), 3)

        # if the right button was clicked, draw a circle
        if event == cv2.EVENT_RBUTTONDOWN:
            cv2.circle(self.image, (x, y), 50, (0, 0, 255), 3)

        # if the middle button was clicked, perform a translation
        if event == cv2.EVENT_MBUTTONDOWN:
            rows, cols = self.image.shape[:2]
            M = np.float32([[1, 0, 100], [0, 1, 50]])
            self.image = cv2.warpAffine(self.image, M, (cols, rows))

    def display_image(self):
        ''' Display the image '''
        cv2.namedWindow(winname='my_image')

        # connect the mouse callback function to the window
        cv2.setMouseCallback('my_image', self.mouse_callback)
        while True:
            cv2.imshow('my_image', self.image)
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
        cv2.destroyAllWindows()

    def display_current_time(self):
        ''' Display the current time on the image '''
        current_time = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        cv2.putText(self.image, str(current_time), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        while True:
            cv2.imshow('my_image', self.image)
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
        cv2.destroyAllWindows()

    def image_processing(self):
        ''' Perform image processing on the image '''
        while True:
            cv2.imshow('my_image', self.image)
            key = cv2.waitKey(1) & 0xFF

            # convert to grayscale
            if key == ord('g'):
                self.image = cv2.cvtColor(self.image, cv2.COLOR_BGR2GRAY)  # convert to grayscale
                self.history.append(self.image.copy())  # add the image to the history

            # convert to rgb
            elif key == ord('r'):
                self.image = self.original_image  # restore the original image

            # save the image
            elif key == ord('s'):
                cv2.imwrite('new_image.jpg', self.image)
                print('Image saved successfully!')

            # crop the image
            elif key == ord('c'):
                self.image = self.image[556:1160, 10:810]  # [width, height]
                self.history.append(self.image.copy())

            # undo the last operation
            elif key == ord('z'):
                if len(self.history) > 0:
                    self.image = self.history.pop()
                else:
                    print('No more operations to undo!')


            elif key == ord('h'):
                help_message = "Operations:\n" \
                            "g - Convert image to grayscale\n" \
                            "r - Convert image to RGB\n" \
                            "s - Save the image\n" \
                            "c - Crop the image\n" \
                            "z - Undo the last operation\n" \
                            "h - Display this help message"
                print(help_message)
            
            # exit
            if key == ord('q'):
                break

        
        cv2.destroyAllWindows()


    def implement_bind_trackbar(self):
        ''' Implement a bind trackbar '''

        def update_image(x):
            ''' Update the image based on the trackbar positions '''
            brightness = cv2.getTrackbarPos('Brightness', 'image')
            contrast = cv2.getTrackbarPos('Contrast', 'image')
            saturation = cv2.getTrackbarPos('Saturation', 'image')
            switch = cv2.getTrackbarPos('Switch', 'image')

            # Apply adjustments only if the switch is on (1)
            if switch == 1:
                # Apply brightness and contrast adjustments
                adjusted_image = cv2.convertScaleAbs(self.original_image, alpha=contrast/100, beta=brightness)

                # Apply saturation adjustment
                hsv_image = cv2.cvtColor(adjusted_image, cv2.COLOR_BGR2HSV)
                hsv_image[:, :, 1] = np.clip(hsv_image[:, :, 1] + saturation, 0, 255)
                adjusted_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
            else:
                adjusted_image = self.original_image.copy()

            cv2.imshow('image', adjusted_image)

        # create a window and trackbars
        cv2.namedWindow('image')
        cv2.createTrackbar('Brightness', 'image', 0, 100, update_image)
        cv2.createTrackbar('Contrast', 'image', 0, 100, update_image)
        cv2.createTrackbar('Saturation', 'image', 0, 100, update_image)
        cv2.createTrackbar('Switch', 'image', 0, 1, update_image)

        # update the image initially
        update_image(0)

        while True:
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break

        cv2.destroyAllWindows()



    


In [3]:
processor = ImageProcessor('./images/image.jpg') # Initialize the class with the image path


1. Load an image from disk using the cv2.imread() function.

In [4]:
processor.read_image()

2. Define a mouse callback function that responds to mouse 
events. The function should perform the following tasks 
based on the type of mouse event:
    - If the left mouse button is pressed and dragged, draw a rectangle on the image using the cv2.rectangle() function.
    - If the right mouse button is pressed and dragged, draw a circle on the image using the cv2.circle() function.
    - If the middle mouse button is pressed and dragged, perform image translation on the image using  the cv2.warpAffine() function.

<br />

3. Display the image with the mouse callback function using the cv2.imshow() function.


In [6]:
processor.display_image()


<img src="./images/task_2_3.png" alt="task" width="400">


4. Define a function to display the current time on the frame. 
You can use Python's built-in datetime module to get the 
current time:
5. Add the current time to the frame using 
the cv2.putText() function.
6. Wait for a key event using the cv2.waitKey() function.


In [7]:
processor.display_current_time()

<img src="./images/task_4_5_6.png" alt="task" width="400">


7. If the 'g' key is pressed, convert the image to grayscale using the cv2.cvtColor() function.
8. If the 'r' key is pressed, reset the image to its original state.
9. If the 's' key is pressed, save the image to disk using  the cv2.imwrite() function.
10. If the 'c' key is pressed, crop the region of interest  defined by the rectangle using array slicing.
11. If the 'z' key is pressed, undo the previous operation.
12. If the 'q' key is pressed, exit the program.
13. If the 'h' key is pressed, display a help message.
14. Implement a history of image modifications so that the 
user can undo previous operations.

In [6]:
processor.image_processing()

Operations:
g - Convert image to grayscale
r - Convert image to RGB
s - Save the image
c - Crop the image
z - Undo the last operation
h - Display this help message


<img src="./images/procced_image.jpg" alt="task" width="400">

In [7]:
processor.implement_bind_trackbar()

<img src="./images/apply_b.png" alt="task" width="800">